## Pipeline Parallelism
- 이번 세션에서는 파이프라인 병렬화에 대해 알아보겠습니다.

### 1. Inter-layer model parallelism
- 파이프라인 병렬화는 Inter-layer 모델 병렬화를 개선한 것입니다.
- Inter-layer 모델 병렬화는 아래와 같이 특정 GPU에 특정 레이어들을 할당하는 모델 병렬화 방법입니다.
- 아래 그림에서는 GPU1번에 1,2,3번 레이어가 할당되었고, GPU2번에 4,5번 레이어가 할당 되었습니다.

![](../images/inter_layer.png)

- 그러나 이전 레이어의 출력을 다음 레이어의 입력으로 하는 신경망의 특성상 특정 GPU의 연산이 끝나야 다른 GPU가 연산을 시작할 수 있습니다.
- 즉, 아래의 그림처럼 Inter-layer 모델 병렬화는 동시에 하나의 GPU만 사용할 수 있다는 치명적인 한계를 가지고 있습니다.

![](../images/inter_layer_2.png)
![](../images/inter_layer_3.gif)


## 2. GPipe
- GPipe는 Google에서 개발된 파이프라인 병렬화 기법으로 Inter Layer 모델 병렬화 시 GPU가 쉬는 시간 (idle time)을 줄이기 위해 등장했습니다.
- GPipe는 mini-batch를 micro-batch로 한번 더 쪼개서 학습 과정을 파이프라이닝 하는 방식으로 동작합니다.

![](../images/gpipe_1.png)

<br>

### Micro-batch
- Mini-batch는 전체 데이터셋을 n개로 분할한 서브샘플 집합입니다.
- Micro-batch는 Mini-batch를 m개로 한번 더 분할한 서브샘플 집합입니다.

![](../images/gpipe_2.png)

<br>

### Pipelining
- GPipe는 미니배치를 마이크로 배치로 쪼개고 연산을 파이프라이닝 합니다.

![](../images/gpipe_3.gif)


## 3. GPipe with PyTorch
- kakaobrain에서 공개한 `torchgpipe`를 사용하면 손쉽게 GPipe를 사용할 수 있습니다.
- 단, `nn.Sequential`로 래핑된 모델만 pipeline parallelism이 가능합니다.
- 모든 모듈의 입력과 출력 타입은 `torch.Tensor` 혹은 `Tuple[torch.Tensor]`로 제한됩니다.

In [ ]:
"""
src/gpipe.py
"""

import torch
import torch.nn as nn
from datasets import load_dataset
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchgpipe import GPipe
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel
from transformers.models.gpt2.modeling_gpt2 import GPT2Block as GPT2BlockBase


class GPT2Preprocessing(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed_dim = config.hidden_size
        self.wte = nn.Embedding(config.vocab_size, self.embed_dim)
        self.wpe = nn.Embedding(config.max_position_embeddings, self.embed_dim)
        self.drop = nn.Dropout(config.embd_pdrop)

    def forward(self, input_ids):
        input_shape = input_ids.size()
        input_ids = input_ids.view(-1, input_shape[-1])
        position_ids = torch.arange(
            0, input_shape[-1], dtype=torch.long, device=input_ids.device
        )
        position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])
        inputs_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)
        hidden_states = inputs_embeds + position_embeds
        hidden_states = self.drop(hidden_states)
        return hidden_states


class GPT2Block(GPT2BlockBase):
    def forward(self, hidden_states):
        hidden_states = super(GPT2Block, self).forward(
            hidden_states=hidden_states,
        )
        return hidden_states[0]


class GPT2PostProcess(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_f = nn.LayerNorm(
            config.hidden_size,
            eps=config.layer_norm_epsilon,
        )
        self.lm_head = nn.Linear(
            config.hidden_size,
            config.vocab_size,
            bias=False,
        )

    def forward(self, hidden_states):
        hidden_states = self.ln_f(hidden_states)
        lm_logits = self.lm_head(hidden_states)
        return lm_logits


def create_model_from_pretrained(model_name):
    pretrained = GPT2LMHeadModel.from_pretrained(model_name)

    preprocess = GPT2Preprocessing(pretrained.config)
    preprocess.wte.weight = pretrained.transformer.wte.weight
    preprocess.wpe.weight = pretrained.transformer.wpe.weight

    blocks = pretrained.transformer.h
    for i, block in enumerate(blocks):
        block.__class__ = GPT2Block
        # 0, 1, 2 => 0
        # 3, 4, 5 => 1
        # 6, 7, 8 => 2
        # 9, 10, 11 => 3

    postprocess = GPT2PostProcess(pretrained.config)
    postprocess.ln_f.weight = pretrained.transformer.ln_f.weight
    postprocess.ln_f.bias = pretrained.transformer.ln_f.bias
    postprocess.lm_head.weight.data = pretrained.lm_head.weight.data.clone()

    return nn.Sequential(preprocess, *blocks, postprocess)


if __name__ == "__main__":
    world_size = 4
    model = create_model_from_pretrained(model_name="gpt2")
    model = GPipe(
        model,
        balance=[4, 3, 3, 4],
        devices=[0, 1, 2, 3],
        chunks=world_size,
    )

    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    datasets = load_dataset("squad").data["train"]["context"]
    datasets = [str(sample) for sample in datasets]
    data_loader = DataLoader(datasets, batch_size=8, num_workers=8)

    optimizer = Adam(model.parameters(), lr=3e-5)
    loss_fn = nn.CrossEntropyLoss()

    for i, data in enumerate(data_loader):
        optimizer.zero_grad()
        tokens = tokenizer(data, return_tensors="pt", truncation=True, padding=True)
        input_ids = tokens.input_ids.to(0)
        labels = tokens.input_ids.to(world_size - 1)

        lm_logits = model(input_ids)
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss = nn.CrossEntropyLoss()(
            shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
        )
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"step: {i}, loss: {loss}")
        if i == 300:
            break


In [4]:
# !python -m torch.distributed.launch --nproc_per_node=4 ../src/gpipe.py
!python ../src/gpipe.py

Reusing dataset squad (/home/ubuntu/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 316.63it/s]
step: 0, loss: 6.088901519775391
step: 10, loss: 3.245741367340088
step: 20, loss: 2.822031021118164
step: 30, loss: 2.5435261726379395
step: 40, loss: 2.8309974670410156
step: 50, loss: 2.3458638191223145
step: 60, loss: 2.5426201820373535
step: 70, loss: 2.24003005027771
step: 80, loss: 2.474609613418579
step: 90, loss: 2.9318113327026367
step: 100, loss: 2.8149759769439697
step: 110, loss: 2.473766803741455
step: 120, loss: 2.9568164348602295
step: 130, loss: 2.4007227420806885
step: 140, loss: 2.9461541175842285
step: 150, loss: 3.9239771366119385
step: 160, loss: 3.027210235595703
step: 170, loss: 3.0363430976867676
step: 180, loss: 1.678962230682373
step: 190, loss: 3.5441555976867676
step: 200, loss: 3.6646995544433594
step: 210, loss: 3.53093743324


## 4. 1F1B Pipelining (PipeDream)

- Microsoft에서 공개한 `PipeDream`은 `GPipe`와는 약간 다른 방식의 파이프라이닝을 수행합니다.
- 흔히 이 방법을 1F1B라고 부르는데, 모든 Forward가 끝나고 나서 Backward를 수행하는 GPipe와 달리 `PipeDream`은 Forward와 Backward를 번갈아가면서 수행합니다.

![](../images/1f1b.png)

- 1F1B Pipelining에는 다음과 같은 두가지 챌린지가 존재합니다.
  - (1) Weight version managing
  - (2) Work partitioning

<br>

### Weight version managinig
- GPipe의 경우 하나의 weight 버전만 운용하지만 주기적으로 Pipeline flush가 일어납니다.
- Pipeline flush란 계산된 Gradient를 통해 파라미터를 업데이트 하는 과정.
- 이러한 flush 과정 중에는 어떠한 forward, backward 연산도 하지 않기 때문에 처리 효율이 떨어집니다.

![](../images/pipeline_flush.png)

- PipeDream은 이러한 flush 없이 여러 버전의 파라미터 상태를 지속적으로 관리합니다.
- 만약 최신버전의 파라미터만 저장하고 있으면 이전 layer의 출력이 다음 layer로 전송될 때, 다음 layer 부분이 업데이트 될 수도 있습니다.

<img src="../images/1f1b.gif" width=800>

- 이러한 문제를 막기 위해 여러 버전의 weight를 저장하여 관리합니다.
- 그러나 여러버전의 weight를 저장하면 메모리 공간을 많이 차지하게 됩니다.
- 따라서 이 부분에서 트레이드 오프가 발생합니다.
  - GPipe: 메모리 효율적, 프로세싱 비효율적
  - PipeDream: 메모리 비효율적, 프로세싱 효율적
  
<br>

### Work Partitioning
- 두번쨰 문제는 뉴럴넷을 어떻게 쪼갤건지에 대한 문제.
- 단순히 Layer별로 쪼개는게 답이라고 할 순 없습니다.
- 각 파티션의 running time이 비슷해야 idle time을 최소화 할 수 있을 것입니다.
- 그 이외에도 parameter size, activation memory 등을 고려해야 합니다.

<img src="../images/pipe_dream.png" width=600>

- PipeDream은 Profiling과 Optimizing을 통해 최적의 Partioning 전략을 찾아냅니다.
- 이는 computing time, parameter size, activation memory 등을 고려합니다.

<br><br>

## 5. Variation of 1F1B Pipelining

- PipeDream의 1F1B 파이프라이닝을 개선한 두가지 버전의 파이프라인 전략을 소개합니다.

<br>

### 1) PipeDream 2BW (2-buffered weight update)
- PipeDream은 메모리 비효율적이였는데, 그것을 해결하기 위해 등장.
- 핵심 아이디어는 파이프라이닝 중에 Gradient Accumulation을 수행하는 것.
- 여러개의 Gradient들을 모아두다가 한번에 업데이트를 수행하는 방식으로 해결.
- 단 두개의 weight version만 유지하면 되고, flush 과정도 필요 없음.

![](../images/pipe_dream_2bw.png)

<br>

### 2) PipeDream Flush: Memory Efficient Pipelining
- 1F1B와 Pipeline Flush를 결합한 파이프라이닝 방법
- GPIpe와 비교하여 idle time은 비슷하나, forward-backward 과정에서 유지해야 하는 activation 메모리가 줄어서 효율적.
- 단일 가중치만 유지하면 되기 때문에 PipeDream 2BW보다도 더 메모리 효율적임.
- **DeepSpeed 파이프라인 병렬처리 모듈이 PipeDream Flush로 구현되어 있음.**
- https://github.com/microsoft/DeepSpeed/issues/1110

![](../images/pipe_dream_flush.png)

![](../images/pipe_dream_flush_2.png)